In [1]:
from importlib import reload
import itertools
import pandas as pd
import utils 
reload(utils)
from utils import *
import numpy as np
from mapreduce import FilteringMapReduce, map_reduce_df

In [2]:
# https://www.kaggle.com/ceshine/mean-baseline-lb-59/code
types = {'id': 'int32',
         'date': 'int16',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float32',
         'onpromotion': bool}
days = get_days_in_range('2017-08-01', '2017-08-15')
cols = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']
mapreduce = FilteringMapReduce(lambda df: df[df.date.isin(days)])

In [3]:
df_train = map_reduce_df('./data/train_processed.csv', mapreduce, types=types, position=120000000, cols=cols, verbose=True)

# df_train = pd.read_csv(
#     './data/train_processed.csv', usecols=[1, 2, 3, 4, 5], dtype=types,
#     converters={'unit_sales': lambda u: float(u) if float(u) > 0 else 0},
#     skiprows=range(1, 124035460)
# )

Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 1570968, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [4]:
df_train.ix[df_train.unit_sales < 0, "unit_sales"] = 0
df_train["unit_sales"] = np.log1p(df_train["unit_sales"])

In [5]:
u_dates = df_train.date.unique()
u_stores = df_train.store_nbr.unique()
u_items = df_train.item_nbr.unique()
df_train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
df_train = df_train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)
# Fill NAs
df_train.loc[:, "unit_sales"].fillna(0, inplace=True)
# Assume missing entris imply no promotion
df_train.loc[:, "onpromotion"].fillna(False, inplace=True)

In [45]:
test.head()

,id,unit_sales
32,125497042,0.900737
33,125707696,0.900737
34,125918350,0.900737
35,126129004,0.900737
36,126339658,0.900737


In [6]:
df_train.reset_index(inplace=True)
df_train = df_train.groupby(['item_nbr', 'store_nbr', 'onpromotion'])['unit_sales'].mean().to_frame('unit_sales')
df_train.head()

In [10]:
!telegram-send "Preparations finished"

In [11]:
test = pd.read_csv("./data/test.csv").set_index(['item_nbr', 'store_nbr', 'onpromotion']).join(df_train, how='left').fillna(0.0).reset_index()

In [12]:
items_cols = pd.read_csv('./data/items_encoded.csv', nrows=1).columns
bit_cols = list(set(items_cols)-{'item_nbr'})
item_types = dict(map(lambda x: (x, bool), bit_cols))
items = pd.read_csv('./data/items_encoded.csv', dtype=item_types)

stores_cols = pd.read_csv('./data/stores_encoded.csv', nrows=1).columns
bit_cols = list(set(stores_cols)-{'store_nbr'})
stores_types = dict(map(lambda x: (x, bool), bit_cols))
stores = pd.read_csv('./data/stores_encoded.csv', dtype=stores_types)

test_ext = test.merge(items, on='item_nbr')
test_ext = test_ext.merge(stores, on='store_nbr')

In [13]:
test_ext['store_item_tuple'] = list(test_ext[['item_nbr', 'store_nbr']].itertuples(index=False))
test_ext.head()

,item_nbr,store_nbr,onpromotion,id,date,unit_sales,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,...,cluster_17,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,store_item_tuple
0,96995,1,False,125497040,2017-08-16,0.312142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(96995, 1)"
1,96995,1,False,125707694,2017-08-17,0.312142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(96995, 1)"
2,96995,1,False,125918348,2017-08-18,0.312142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(96995, 1)"
3,96995,1,False,126129002,2017-08-19,0.312142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(96995, 1)"
4,96995,1,False,126339656,2017-08-20,0.312142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(96995, 1)"


In [14]:
n_train = pd.read_csv('./data/n_train.csv')
n_train_tuples = list(n_train.itertuples(index=False))
test_known = test_ext[~test_ext.store_item_tuple.isin(n_train_tuples)]
test_unknown = test_ext[test_ext.store_item_tuple.isin(n_train_tuples)]
del test_ext

In [15]:
import lightgbm as lgb
from scipy.sparse import csr_matrix
from metric import get_weights

In [24]:
x_cols = list(set(test_known.columns) - {'item_nbr', 'store_nbr', 'id', 'date', 'unit_sales', 'store_item_tuple'})
split_point = int(len(test_known)*0.7)

train = test_known.iloc[:split_point]
weights_train = get_weights(train['item_nbr'])
data_train = csr_matrix(train[x_cols]).asfptype()
label_train = train['unit_sales']
dataset_train = lgb.Dataset(data_train, label_train, weight=weights_train)

validation = test_known.iloc[split_point:]
weights_validation = get_weights(validation['item_nbr'])
data_validation = csr_matrix(validation[x_cols]).asfptype()
label_validation = validation['unit_sales']
dataset_validation = lgb.Dataset(data_validation, label_validation, weight=weights_validation)

In [28]:
param = {'num_leaves':30, 'num_trees':100, 'objective':'regression_l2', 'metric':'mse'}
num_round = 20
bst = lgb.train(param, dataset_train, num_round, valid_sets=dataset_validation, early_stopping_rounds=10, verbose_eval=True)

C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\lightgbm\engine.py:98: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 1.13396
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 1.11429
[3]	valid_0's l2: 1.10337
[4]	valid_0's l2: 1.09079
[5]	valid_0's l2: 1.08
[6]	valid_0's l2: 1.07336
[7]	valid_0's l2: 1.06445
[8]	valid_0's l2: 1.05964
[9]	valid_0's l2: 1.05227
[10]	valid_0's l2: 1.04841
[11]	valid_0's l2: 1.04194
[12]	valid_0's l2: 1.03861
[13]	valid_0's l2: 1.03412
[14]	valid_0's l2: 1.03191
[15]	valid_0's l2: 1.02434
[16]	valid_0's l2: 1.01713
[17]	valid_0's l2: 1.01333
[18]	valid_0's l2: 1.01182
[19]	valid_0's l2: 1.0106
[20]	valid_0's l2: 1.00791
[21]	valid_0's l2: 1.00578
[22]	valid_0's l2: 1.0084
[23]	valid_0's l2: 1.00557
[24]	valid_0's l2: 1.00339
[25]	valid_0's l2: 1.00305
[26]	valid_0's l2: 1.00178
[27]	valid_0's l2: 1.00371
[28]	valid_0's l2: 1.00205
[29]	valid_0's l2: 1.00005
[30]	valid_0's l2: 0.99857
[31]	valid_0's l2: 0.997872
[32]	valid_0's l2: 0.996896
[33]	valid_0's l2: 0.995361
[34]	valid_0's l2: 0.995094
[35]	valid_0's l2: 0.992311
[3

In [27]:
!telegram-send "LightGBM done"

In [29]:
test_unknown['unit_sales'] = bst.predict(test_unknown[x_cols])
test_unknown.head()

,item_nbr,store_nbr,onpromotion,id,date,unit_sales,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,...,cluster_17,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,store_item_tuple
32,103501,1,False,125497042,2017-08-16,0.900737,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(103501, 1)"
33,103501,1,False,125707696,2017-08-17,0.900737,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(103501, 1)"
34,103501,1,False,125918350,2017-08-18,0.900737,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(103501, 1)"
35,103501,1,False,126129004,2017-08-19,0.900737,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(103501, 1)"
36,103501,1,False,126339658,2017-08-20,0.900737,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"(103501, 1)"


In [41]:
submission = pd.read_csv('./data/sample_submission.csv')
del submission['unit_sales']
#test = pd.concat([test_unknown[['id','unit_sales']], test_known[['id','unit_sales']]])
#submission = submission.merge(test, on='id', how='left')
submission = submission.merge(test_known[['id','unit_sales']], on='id', how='left')
submission.fillna(0.0, inplace=True)
submission.ix[submission['unit_sales'] < 0, 'unit_sales'] = 0
submission['unit_sales'] = np.expm1(submission['unit_sales'])
submission.head()

,id,unit_sales
0,125497040,0.366349
1,125497041,0.212509
2,125497042,0.000000
3,125497043,0.873407
4,125497044,1.915161


In [42]:
submission.to_csv('./submissions/kernel_mean_baseline3.csv.gz', float_format='%.3f', index=None, compression="gzip")

In [44]:
!telegram-send "Submission is done"